<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/geemap/tree/master/examples/template/template.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/geemap/blob/master/examples/template/template.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/template/template.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Clasificación supervisada SVM
Script trabajado por Francisco Albornoz (2021-04-28 12:51:11 ) / Última modificación / Análisis y clasificación supervisada región de Aysen

<img src=https://www.ecopaysen.cl/wp-content/uploads/2019/12/ecopaysen_marca_web.png>

In [ ]:
# Installs geemap package
# import subprocess

# try:
#     import geemap
# except ImportError:
#     print("Installing geemap ...")
#     subprocess.check_call(["python", "-m", "pip", "install", "geemap"])
# !pip install scikit_learn


In [1]:
import os
import ee
import geemap
import xarray_leaflet
import rasterio 
from rasterio import plot
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import pandas as pd

## Crear un mapa interactivo 
The default basemap is `Google Maps`. [Additional basemaps](https://github.com/giswqs/geemap/blob/master/geemap/basemaps.py) can be added using the `Map.add_basemap()` function. 

In [2]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

## Escogemos la imagen landsat 

In [3]:
centroid = ee.Geometry.Point([-72, -44])

landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(centroid) \
    .filterDate('2020-01-01', '2021-03-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

landsat_vis = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(centroid, 8)
Map.addLayer(landsat, landsat_vis, "Landsat-8")


## Importar rasters locales 

In [4]:
%pwd

'/mnt/c/users/usuario/Documents/GitHub/VitoAlbornoz/GEE'

In [5]:
# verificar en el sistema del pc cual es la ruta
# os.path.expanduser('/mnt/c/Users/R7 3700X/Documents/GitHub/VitoAlbornoz/') 
# os.path.expanduser('/mnt/c/Users/Usuario/Documents/Github/Vitoalbornoz/')

In [6]:
# verificar en el sistema del pc cual es la ruta
out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/'), 'Data')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

raster_aysen = os.path.join(out_dir, 'landcover_2018_aysen.tif')

In [7]:
out_dir

'/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/Data'

In [8]:
raster_aysen

'/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/Data/landcover_2018_aysen.tif'

In [9]:
Map.add_raster(raster_aysen, colormap= 'terrain', layer_name='Landcover 2018')

In [10]:
landcover = ee.Image("users/franciscoalbornoz/landcover_sur")
landcover

In [11]:
raster = rasterio.open(out_dir+'/landcover_2018_aysen.tif')

In [12]:
raster.bounds

BoundingBox(left=-73.169644655, bottom=-46.394042378, right=-70.719691266, top=-43.705745621)

In [13]:
raster.crs

CRS.from_epsg(4326)

In [14]:
raster

<open DatasetReader name='/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/Data/landcover_2018_aysen.tif' mode='r'>

In [15]:
mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")

landsat_vis = {
    'min': 0,
    'max': 1,
    'bands': ['b5', 'b4', 'b3']
}
Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")

Map

Map(center=[-45.0498939995, -71.9446679605], controls=(WidgetControl(options=['position'], widget=HBox(childre…

## Obteniendo los valores para el Training Datasets

In [50]:
work_dir = os.path.expanduser('../Data/shapes/')

merged_shp = os.path.join(work_dir, 'roi_h.shp')
val_shp= os.path.join(work_dir, 'puntos_validacion.shp')

In [51]:
merged = geemap.shp_to_ee(merged_shp)
Map.addLayer(merged, {}, 'Áreas de entrenamiento')

In [55]:
val = geemap.shp_to_ee(val_shp)
Map.addLayer(val, {}, 'Áreas de Validación')

In [53]:
dataset = mosaico.sampleRegions(**{
    'collection': merged,
    'properties': ['IDH'],
    'scale' : 30 ,
})

In [56]:
datasetval = mosaico.sampleRegions(**{
    'collection': val,
    'properties': ['IDH'],
    'scale': 30,
})

In [57]:
print(dataset.size().getInfo())

14099


In [58]:
print(datasetval.size().getInfo())

328


In [59]:
dataset.propertyNames()

In [60]:
datasetval.propertyNames()

In [61]:
print(dataset.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'IDH': 13, 'b1': 0.026168961077928543, 'b2': 0.04253531992435455, 'b3': 0.03470490872859955, 'b4': 0.33766427636146545, 'b5': 0.1495322287082672, 'b6': 0.06873124837875366, 'b7': 0.8135994672775269, 'b8': 0.7762474417686462, 'b9': 0.5209251642227173}}


In [62]:
print(datasetval.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'IDH': 1, 'b1': 0.01979999989271164, 'b2': 0.017999999225139618, 'b3': 0.010499999858438969, 'b4': 0.01730000041425228, 'b5': 0.00989999994635582, 'b6': 0.0071000000461936, 'b7': 0.24460433423519135, 'b8': -0.019829994067549706, 'b9': 0.019325504079461098}}


In [63]:
out_csv = os.path.join(out_dir, 'Dataset_entrenamiento.csv')
geemap.extract_values_to_points(merged, mosaico, out_csv)

Generating URL ...
Please wait ...
Data downloaded to /mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/Data/Dataset_entrenamiento.csv


In [64]:
out_csv2 =os.path.join(out_dir, 'Dataset_validación.csv')
geemap.extract_values_to_points(val, mosaico, out_csv2)

Generating URL ...
Please wait ...
An error occurred while downloading. 
 Retrying ...
Generating URL ...
Please wait ...
Data downloaded to /mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/Data/Dataset_validación.csv


In [65]:
df = pd.read_csv(out_csv)

In [66]:
df

,path,RF,CLASS_CLRS,FILL_CLASS,POLY_TYPE,ID,IDH,CLASS_ID,CLASS_NAME,layer,b2,b3,b4,b5,b6,b7,b8,b9,system:index,b1
0,E:/Carpeta de Trabajo (Fco)/Datos para github/...,ACHAPARRADO,"0,205,0",1,5,1,13,0,ACHAPARRAD,achaparrado_2018_2210_grande,0.042535,0.034705,0.337664,0.149532,0.068731,0.813599,0.776247,0.520925,0_0,0.026169
1,E:/Carpeta de Trabajo (Fco)/Datos para github/...,ACHAPARRADO,"0,205,0",1,5,1,13,0,ACHAPARRAD,achaparrado_2018_2210_grande,0.039025,0.030058,0.313210,0.130905,0.061290,0.824872,0.778417,0.503669,0_1,0.024665
2,E:/Carpeta de Trabajo (Fco)/Datos para github/...,ACHAPARRADO,"0,205,0",1,5,1,13,0,ACHAPARRAD,achaparrado_2018_2210_grande,0.034077,0.025750,0.324303,0.129287,0.055219,0.852881,0.809826,0.526826,0_2,0.018188
3,E:/Carpeta de Trabajo (Fco)/Datos para github/...,ACHAPARRADO,"0,205,0",1,5,1,13,0,ACHAPARRAD,achaparrado_2018_2210_grande,0.038736,0.027363,0.350001,0.148841,0.065440,0.854979,0.800706,0.551604,0_3,0.022684
4,E:/Carpeta de Trabajo (Fco)/Datos para github/...,ACHAPARRADO,"0,205,0",1,5,1,13,0,ACHAPARRAD,achaparrado_2018_2210_grande,0.036138,0.026296,0.328300,0.131833,0.057624,0.851684,0.801680,0.530082,0_4,0.021386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14094,E:/Carpeta de Trabajo (Fco)/Datos para github/...,SUELO DES,"255,255,0",1,5,18,11,4,BARBECHO3,suelo_desnudo_agricola_2018_0310_grande,0.061346,0.058328,0.325896,0.180328,0.089545,0.696385,0.683163,0.453903,936_15,0.028166
14095,E:/Carpeta de Trabajo (Fco)/Datos para github/...,SUELO DES,"255,255,0",1,5,18,11,4,BARBECHO3,suelo_desnudo_agricola_2018_0310_grande,0.065845,0.065742,0.318638,0.191173,0.096099,0.657933,0.657487,0.428938,936_16,0.032137
14096,E:/Carpeta de Trabajo (Fco)/Datos para github/...,SUELO DES,"255,255,0",1,5,18,11,4,BARBECHO3,suelo_desnudo_agricola_2018_0310_grande,0.067506,0.066192,0.318410,0.193196,0.096287,0.655789,0.650152,0.427680,936_17,0.031339
14097,E:/Carpeta de Trabajo (Fco)/Datos para github/...,SUELO DES,"255,255,0",1,5,18,11,4,BARBECHO3,suelo_desnudo_agricola_2018_0310_grande,0.068516,0.067344,0.329149,0.199751,0.098366,0.660301,0.655408,0.438048,936_18,0.032496


In [67]:
df2 = pd.read_csv(out_csv2)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 2


In [68]:
df2

,cobertura,OBJECTID_1,ID,IDH,layer,b2,b3,b4,b5,b6,b7,b8,b9,system:index,b1
0,agua,10.0,NaN,1,02_agua_val_subset,0.018000,0.010500,0.017300,0.009900,0.007100,0.244604,-0.019830,0.019326,0_0,0.019800
1,agua,167.0,NaN,1,02_agua_val_subset,0.022920,0.016848,0.187948,0.065009,0.031107,0.835466,0.782612,0.364149,2_0,0.010910
2,agua,168.0,NaN,1,02_agua_val_subset,0.021857,0.013616,0.026286,0.016476,0.011934,0.317549,0.091999,0.035203,3_0,0.021381
3,agua,170.0,NaN,1,02_agua_val_subset,0.020023,0.013502,0.025011,0.014753,0.010921,0.298824,0.110768,0.032057,5_0,0.020570
4,agua,171.0,NaN,1,02_agua_val_subset,0.021874,0.015821,0.029951,0.018179,0.012739,0.308702,0.155859,0.038835,6_0,0.024914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,vegetacion altura,NaN,0.0,14,17_vegetacion_altura_val,0.055472,0.054559,0.260036,0.168895,0.094470,0.653148,0.648362,0.378367,458_0,0.031737
324,vegetacion altura,NaN,0.0,14,17_vegetacion_altura_val,0.057084,0.057905,0.236421,0.192860,0.112964,0.606527,0.611018,0.337109,459_0,0.037315
325,vegetacion altura,NaN,0.0,14,17_vegetacion_altura_val,0.059564,0.060805,0.285315,0.192619,0.106765,0.648650,0.654579,0.398012,461_0,0.035541
326,vegetacion altura,NaN,0.0,14,17_vegetacion_altura_val,0.060853,0.060434,0.270101,0.213666,0.121309,0.634328,0.632257,0.378673,462_0,0.036502


### Dividiendo en muestras y datos de validación

In [69]:
# Guardamos las bandas
bands = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']

# Esta propiedad guarda las clases de landcover
label = 'IDH'

# Overlay the points on the imagery to get training.
# sample = landsat.select(bands).sampleRegions(**{
#   'collection': points,
#   'properties': [label],
#   'scale': 30
# })

# Adds a column of deterministic pseudorandom numbers. 
# sample = dataset.randomColumn()

# split = 0.7 

# training = sample.filter(ee.Filter.lt('random', split))
# validation = sample.filter(ee.Filter.gte('random', split))

In [ ]:
# training.first().getInfo()

In [ ]:
# validation.first().getInfo()

## Entrenando el modelo 

In [ ]:
# from sklearn import svm
# from sklearn import ensemble

In [ ]:
# feature_names = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']
# label = "IDH"

In [ ]:
# X = df[feature_names]
# y= df[label]

In [70]:
classifi = ee.Classifier.libsvm('Voting','C_SVC','RBF', gamma = 0.5 , cost = 10 )

In [ ]:
# clf =svm.SVC()
# clf.fit(X, y)

In [71]:
trained = classifi.train(dataset, label, bands)
classified = mosaico.classify(trained)

In [72]:
trained

In [73]:
classified

### Clasificando la imagen

In [ ]:
class_values = [1,2,3,4,5,6,7,8,9,10,11,12,13
    ]
class_palette = ['0c40f1', #1 agua*
 'f0ff00', #2 cultivo*
 'f99504', #3 estepa*
 '17d4c6', #4 humedal*
 '24ff00', #5 matorral abierto*
 '2de50f', #6 matorral arborescente*
 'ccba7d', #7 plantaciones*
 'dec61a', #8 praderas
 '28d909', #9 primario
 'dec61a', #10 secundario
 '28d909', #11 suelo desnudo
 '4e8c44', #12 suelo desnudo altura
 '22c129', #13* vegetacion altura*
]
# class_palette =  
# ['22c129',achaparrado
#  'f0ff00',agricola 
#  '0c40f1', agua
#  '685205',arenal roca
#  '0d4404',bosque degradado
#  'e82d13',cuncuna
#  'f99504',estepa
#  '17d4c6', humedal
#  '1c6330',mallin
#  '24ff00',matorral abierto
#  '2de50f', matorral denso
#  'ccba7d',plantacion
#  'dec61a',praderas
#  '28d909',primario
#  '03c2ff',rios
#  '4e8c44',secundario
#  'fdfdfd', sombra topo
#  'c0c233' suelo desnudo
# ]


In [ ]:
land = classified.set('classification_class_values', class_values)
landcoverf = land.set('classification_class_palette', class_palette)

In [ ]:
Map.addLayer(landcoverf, {}, 'Land cover SVM')
Map

### Categorizando según las clases 

In [ ]:
# landcover.getInfo()

In [ ]:
# class_values = [1,
#  2,
#  3,
#  4,
#  5,
#  6,
#  7,
#  8,
#  9,
#  10,
#  11,
#  12,
#  13,
#  14,
#  15,
#  16]

In [ ]:
# class_palette = ['476ba1',
#  'd1defa',
#  'decaca',
#  'd99482',
#  'ee0000',
#  'ab0000',
#  'b3aea3',
#  '68ab63',
#  '1c6330',
#  'b5ca8f',
#  'a68c30',
#  'ccba7d',
#  'e3e3c2',
#  'caca78',
#  '99c247',
#  '78ae94']


In [ ]:
# landcoverf = result.set('classification_class_values', class_values)
# landcoverf = landcoverf.set('classification_class_palette', class_palette)

In [ ]:
# Map.addLayer(landcoverf, {}, 'Land cover clasificado' )
# Map

### Visualizando el resultado


In [ ]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))


In [ ]:
Map

### Agregando título al mapa

In [ ]:
# Map.add_legend('Clasificación supervisada CART')
# Map

### Evaluación de presición / Accuracy assessment

In [ ]:
train_accuracy = trained.confusionMatrix()

In [ ]:
train_accuracy.getInfo()

In [ ]:
train_accuracy.accuracy().getInfo()

In [ ]:
train_accuracy.kappa().getInfo()

In [ ]:
train_accuracy.producersAccuracy().getInfo()

In [ ]:
train_accuracy.consumersAccuracy().getInfo()

#### Set de datos de validación

In [ ]:
validated = datasetval.classify(trained)

In [ ]:
validated.first().getInfo()

In [ ]:
test_accuracy = validated.errorMatrix('IDH', 'classification')

In [ ]:
test_accuracy.getInfo()

In [ ]:
test_accuracy.accuracy().getInfo()

In [ ]:
test_accuracy.kappa().getInfo()

In [ ]:
test_accuracy.producersAccuracy().getInfo()

In [ ]:
test_accuracy.consumersAccuracy().getInfo()

### Descargando la matrix

In [ ]:
# import csv
# import os

# out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/R7 3700X/'), 'Desktop')
# training_csv = os.path.join(out_dir, 'train_accuracy.csv')
# testing_csv = os.path.join(out_dir, 'test_accuracy.csv')

# with open(training_csv, "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerows(train_accuracy.getInfo())
    
# with open(testing_csv, "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerows(test_accuracy.getInfo())

### Exportando 

In [ ]:
# import os
# out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/R7 3700X/'), 'Desktop')
# out_file = os.path.join(out_dir, 'landcover.tif')

In [ ]:
# geemap.ee_export_image(landcover, filename=out_file, scale=900)


In [ ]:
# geemap.ee_export_image_to_drive(landcover, description='landcover', folder='export', scale=900)